In [56]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [57]:
#Data from https://www.kaggle.com/jessemostipak/hotel-booking-demand
df = pd.read_csv('hotel_bookings.csv')

## EDA

In [58]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [60]:
#get dummy variables for for object columns
cols = ['hotel', 'meal', 'country', 'market_segment',
       'distribution_channel', 'reserved_room_type', 'assigned_room_type',
       'deposit_type', 'customer_type']
df = pd.get_dummies(df, columns=cols)

In [61]:
#see the balance of target classes
df.is_canceled.value_counts()

0    75166
1    44224
Name: is_canceled, dtype: int64

In [62]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
is_canceled,119390.0,0.370416,0.482918,0.0,0.0,0.0,1.0,1.0
lead_time,119390.0,104.011416,106.863097,0.0,18.0,69.0,160.0,737.0
arrival_date_year,119390.0,2016.156554,0.707476,2015.0,2016.0,2016.0,2017.0,2017.0
arrival_date_week_number,119390.0,27.165173,13.605138,1.0,16.0,28.0,38.0,53.0
arrival_date_day_of_month,119390.0,15.798241,8.780829,1.0,8.0,16.0,23.0,31.0
...,...,...,...,...,...,...,...,...
deposit_type_Refundable,119390.0,0.001357,0.036811,0.0,0.0,0.0,0.0,1.0
customer_type_Contract,119390.0,0.034140,0.181590,0.0,0.0,0.0,0.0,1.0
customer_type_Group,119390.0,0.004833,0.069351,0.0,0.0,0.0,0.0,1.0
customer_type_Transient,119390.0,0.750591,0.432673,0.0,1.0,1.0,1.0,1.0


In [63]:
#Set up a correlation table for correlations greater than absolute value of .1
corr_df = df.corr().reset_index()
corr_df = corr_df[corr_df.columns[:2]]
corr_df.columns = ['Column', 'Correlation']
corr_df[abs(corr_df['Correlation']) > .1].sort_values(by='Correlation')

,Column,Correlation
239,deposit_type_No Deposit,-0.477911
19,total_of_special_requests,-0.234658
18,required_car_parking_spaces,-0.195498
207,market_segment_Direct,-0.154390
213,distribution_channel_Direct,-0.151620
13,booking_changes,-0.144381
21,hotel_Resort Hotel,-0.136531
230,assigned_room_type_D,-0.128036
245,customer_type_Transient-Party,-0.124135
83,country_FRA,-0.118254


## Logistic Regression

In [64]:
#Create set of predictor variables and target along with test train split
X_cols = corr_df[abs(corr_df['Correlation']) > .1].sort_values(by='Correlation').Column.unique()
X = df[X_cols]
X.drop('is_canceled', axis=1 ,inplace=True)
y = df.is_canceled
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.3, random_state=42)

/Users/nickthomas/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [65]:
#Fit a logistic Regression model
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

/Users/nickthomas/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7833630478743134

In [66]:
#Create coefficents table
coef_df = pd.DataFrame()
coef_df['Column'] = X_train.columns
coef_df['Coefficient'] = lr.coef_[0]
coef_df.sort_values(by='Coefficient')

,Column,Coefficient
2,required_car_parking_spaces,-2.871986
0,deposit_type_No Deposit,-2.165805
11,country_DEU,-1.324947
10,country_GBR,-0.806987
8,customer_type_Transient-Party,-0.745518
9,country_FRA,-0.705268
1,total_of_special_requests,-0.541306
6,hotel_Resort Hotel,-0.392359
5,booking_changes,-0.305190
7,assigned_room_type_D,-0.134772


In [67]:
#Map model predictions onto test data along with certainty levels
test_df = X_test
real = y_test
predict = lr.predict(X_test)
not_canceled = []
canceled = []
for x in lr.predict_proba(X_test):
    not_canceled.append(x[0])
    canceled.append(x[1])
test_df['is_canceled_real'] = real
test_df['is_canceled_predict'] = np.array(predict)
test_df['probability_not_canceled'] = np.array(not_canceled)
test_df['probability_canceled'] = np.array(canceled)

<ipython-input-67-92fc501bc1ff>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['is_canceled_real'] = real
<ipython-input-67-92fc501bc1ff>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['is_canceled_predict'] = np.array(predict)
<ipython-input-67-92fc501bc1ff>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [68]:
#Merge test dataframe with original dataframe
predicted_df = pd.merge(df, test_df[test_df.columns[-4:]], left_index=True ,right_index=True)
predicted_df.head()

,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,is_canceled_real,is_canceled_predict,probability_not_canceled,probability_canceled
0,0,342,2015,July,27,1,0,0,2,0.0,...,0,0,0,0,1,0,0,0,0.523664,0.476336
1,0,737,2015,July,27,1,0,0,2,0.0,...,0,0,0,0,1,0,0,1,0.148042,0.851958
3,0,13,2015,July,27,1,0,1,1,0.0,...,0,0,0,0,1,0,0,0,0.938317,0.061683
4,0,14,2015,July,27,1,0,2,2,0.0,...,0,0,0,0,1,0,0,0,0.875052,0.124948
6,0,0,2015,July,27,1,0,2,2,0.0,...,0,0,0,0,1,0,0,0,0.738974,0.261026


In [69]:
#Pair down columns to prior to creating dummy variables
cols = ['is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled',
    'booking_changes', 'agent',
       'company', 'days_in_waiting_list', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', 'is_canceled_real', 'is_canceled_predict',
       'probability_not_canceled', 'probability_canceled']
predicted_df = predicted_df[cols]

In [70]:
predicted_df.to_excel('predicted_cases.xlsx')